In [4]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
from qiskit.circuit.library import UnitaryGate
import math 
import numpy as np

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol without an attacker.



In [14]:
class CustomCircuit(QuantumCircuit):
    W_gate = UnitaryGate((1/np.sqrt(2))*np.array([[1, 1],[1, -1]], dtype=complex),label="W")
    
    V_gate = UnitaryGate((1/np.sqrt(2))*np.array([[-1, 1],[1, 1]], dtype=complex),label="V")
    
    def w(self, q):
        self.append(self.W_gate, [q])
        return self

    def v(self, q):
        self.append(self.V_gate, [q])
        return self

class RandomCircuit(QuantumCircuit):
    
    third_prob_gate = UnitaryGate((1/np.sqrt(3))*np.array([[1, -np.sqrt(2)],[np.sqrt(2), 1]], dtype=complex),label="P")
    
    def p(self,q):
        self.append(self.third_prob_gate, [q])
        return self

def n_even(N):
    if N%2!=0:
        N+=1
    return N

def initialize_entangled_circuit():
    qc = CustomCircuit(2,2)
    
    qc.h(0)
    qc.cx(0,1)
    qc.x(1)
    qc.z(0)

    return qc

def random_number_3():
    sim = BasicSimulator()
    qc = RandomCircuit(2,2)
    qc.p(0)
    qc.h(1)
    qc.measure([0,1],[0,1])

    transpiled = transpile(qc,sim)

    bits = sim.run(transpiled, shots=1, memory=True).result().get_memory()[0]
    
    bits = bits[::-1]

    if bits[0] == "0":
        return 0
    else:
        if bits[1] == "0":
            return 1
        else:
            return 2


Counter({0: 3039, 2: 2998, 1: 2963})
{0: 0.33766666666666667, 1: 0.32922222222222225, 2: 0.33311111111111114}


In [7]:
A = [
    lambda q,j: q.h(j), 
     lambda q,j: q.w(j), 
     lambda q,j: None
]

B = [
    lambda q,j: q.w(j),
     lambda q,j: None,
     lambda q,j: q.v(j)
]

sim = BasicSimulator()
N=30
N=n_even(N)
alice_choices = []
bob_choices = []
matches = []


for i in range(9*N//4):
    
    cs = initialize_entangled_circuit()
    a_choice = random_number_3()
    b_choice = random_number_3()

    alice_choices.append(a_choice)
    bob_choices.append(b_choice)

    if (a_choice == 1 and b_choice == 0) or (a_choice == 2 and b_choice == 1):
        matches.append(i)
        
    
    A[a_choice](cs,0)
    B[b_choice](cs,1)
    cs.measure([0,1],[0,1])


    
   
    transpiled = transpile(cs,sim)
    
    bits = sim.run(transpiled, shots=1, memory=True).result().get_memory()[0]
    


N=30
N=n_even(N)
alice_choices = []
bob_choices = []
alice_meas=[]
bob_meas=[]
matches = []


for i in range(9*N//4):
    
    cs = initialize_entangled_circuit()
    a_choice = random_number_3()
    b_choice = random_number_3()

    alice_choices.append(a_choice)
    bob_choices.append(b_choice)

    if (a_choice == 1 and b_choice == 0) or (a_choice == 2 and b_choice == 1):
        matches.append(i)
        
    
    A[a_choice](cs,0)
    B[b_choice](cs,1)
    cs.measure([0],[0])
    cs.measure([1],[1])

    transpiled = transpile(cs,sim)
    bits = sim.run(transpiled, shots=1, memory=True).result().get_memory()[0]

    alice_meas.append(bits[1])
    bob_meas.append(bits[0])


